In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import datetime

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
train = pd.read_csv("dataset/rand/train_rand_noout3.csv")
val = pd.read_csv("dataset/rand/train_val1_rand.csv")

We prepare the target values for all the sets

In [5]:
y=train['NumberOfSales']
y_val1=val['NumberOfSales']

We prepare the training input samples for all the sets

In [14]:
X=train.drop(['StoreID','Date', 'NumberOfSales', 'Region','Unnamed: 0','Unnamed: 0.1'], axis=1)
X_val1=val.drop(['StoreID','Date', 'NumberOfSales', 'Region','Unnamed: 0'], axis=1)

We create a table to save the results for the search of the hyperparameters

In [9]:
res = {'number of trees' : [], 'number of attributes' : [], 'depth' : [], 'mae' : []}

In [15]:
nt=100
for na in range(20,30,4):
    for depth in range(31,50,8):
        forest = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=nt, max_features=na, n_jobs=-1)
        forest.fit(X, y)
        y_pred=forest.predict(X_val1)
        mae_val1=mean_absolute_error(y_val1,y_pred)
        res['number of trees'].append(nt)
        res['number of attributes'].append(na)
        res['depth'].append(depth)
        res['mae'].append(mae_val1)
        print(mae_val1, na, nt, depth)
r_df = pd.DataFrame(res)

374.6200006977433 20 100 31
375.11330313601843 20 100 39
374.26427971510196 20 100 47
375.0838752046062 24 100 31
375.3403512024537 24 100 39
375.47234870656087 24 100 47
376.62926914530215 28 100 31
376.132221021551 28 100 39
376.45713163575283 28 100 47


In [16]:
r_df['mae'].min()

374.26427971510196

In [17]:
r_df.to_csv("parameters.csv")

Run on the test set and save out in the format for the evaluation of the error

In [18]:
test = pd.read_csv("dataset/rand/train_val2_rand.csv")

In [19]:
y_test = test['NumberOfSales']
X_test=test.drop(['StoreID','Date', 'NumberOfSales', 'Region','Unnamed: 0'], axis=1)

In [20]:
forest = RandomForestRegressor(max_depth=40, random_state=0, n_estimators=201, max_features=21, n_jobs=-1)
forest.fit(X, y)
y_pred_test=forest.predict(X_test)

In [21]:
columns=['Date', 'StoreID', 'RegionID', 'SalesPredicted', 'SalesReal']
index=range(y_test.shape[0])
result=pd.DataFrame(index=index,columns=columns)
result['Date']=test['Date']
result['StoreID']=test['StoreID']
result['RegionID']=test['Region']
result['SalesPredicted']=y_pred_test
result['SalesReal']=y_test

In [22]:
result.head(5)

,Date,StoreID,RegionID,SalesPredicted,SalesReal
0,2018-01-01,1000,7,7344.238806,8540
1,2018-01-02,1000,7,6522.771144,10364
2,2018-01-03,1000,7,6590.437811,4676
3,2018-01-05,1000,7,7281.417910,6267
4,2018-01-06,1000,7,6666.995025,5953


In [23]:
result.to_csv("results.csv")

In [24]:
mean_absolute_error(y_pred_test,y_test)

684.8691719307502